In [ ]:
!pip install ultralytics

In [2]:
import zipfile
import os

data_dir = '/content/SOURCE5'
os.makedirs(data_dir, exist_ok=True)

with zipfile.ZipFile('/content/drive/MyDrive/Datasets/SOURCE5/train/batch1.zip', 'r') as f:
  f.extractall(data_dir)

In [3]:
model_dir = '/content/model'
os.makedirs(model_dir, exist_ok=True)

with zipfile.ZipFile('/content/drive/MyDrive/Models/words-yolov5/words-yolo.zip', 'r') as f:
  f.extractall(model_dir)

In [ ]:
from ultralytics import YOLO
import cv2


model = YOLO('/content/model/best.pt')
model.overrides['data'] = '/content/model/yolo_config.yaml'

image_dir = '/content/SOURCE5/Pages'
output_dir = '/content/annotated'

os.makedirs(output_dir, exist_ok=True)

image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]

for img_file in image_files:
    img_path = os.path.join(image_dir, img_file)

    img = cv2.imread(img_path)

    results = model.predict(source=img, imgsz=946, device='cpu', conf=0.25)
    results = results[0]

    annotations = []

    img_id = os.path.splitext(img_file)[0]

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        class_id = int(box.cls[0])

        img_width, img_height = img.shape[1], img.shape[0]
        x_center = (x1 + x2) / 2 / img_width
        y_center = (y1 + y2) / 2 / img_height
        width = (x2 - x1) / img_width
        height = (y2 - y1) / img_height

        annotations.append(f"{class_id} {x_center} {y_center} {width} {height}\n")

    annotation_file_path = os.path.join(output_dir, f"{img_id}.txt")
    with open(annotation_file_path, 'w') as f:
        f.writelines(annotations)

print(f"Annotations have been saved to {output_dir}.")


In [6]:
import shutil
shutil.make_archive('/content/batch1_annotations', 'zip', '/content/annotated')

'/content/batch1_annotations.zip'